In [1]:
## initialize the variables 
ip = 'aiorc.cqgrwtaxaib2.us-east-1.rds.amazonaws.com'         ## host of the oracle RDS instance
port = 1521                                                  ## port of the oracle RDS instance
SID = 'ORCL'  
username = 'B4B4KE4'
password = 'AI64732661(('
Data_table="IRIS"
PATH="https://a02.s3.ca-central-1.amazonaws.com/ojdbc7.jar"
JdbcOracleUrl = 'jdbc:oracle:thin:{USER_NAME}/{PASSWORD}@//{URL}:{PORT}/{SID}'.format(USER_NAME=username,PASSWORD=password,URL=ip,PORT=port,SID=SID)

In [2]:
import os
import boto3

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

import sagemaker
from sagemaker import get_execution_role
import sagemaker_pyspark

role = get_execution_role()

# Configure Spark to use the SageMaker Spark dependency jars
jars = sagemaker_pyspark.classpath_jars()

classpath = ":".join(sagemaker_pyspark.classpath_jars())
classpath=classpath+":./ojdbc7.jar:./postgresql-42.1.0.jar"
# See the SageMaker Spark Github to learn how to connect to EMR from a notebook instance
spark = SparkSession.builder.config("spark.driver.extraClassPath", classpath)\
    .master("local[*]").getOrCreate()
    
spark

In [13]:
#classpath

In [4]:
JdbcOracleUrl = 'jdbc:oracle:thin:{USER_NAME}/{PASSWORD}@//{URL}:{PORT}/{SID}'.format(USER_NAME=username,PASSWORD=password,URL=ip,PORT=port,SID=SID)

TABLEDF = spark.read.format("jdbc").option("url",JdbcOracleUrl)\
            .option("dbtable",Data_table).option("user",username)\
            .option("password",password)\
            .load()

In [5]:
TABLEDF.show(10)

+------------+-----------+------------+-----------+-------+
|SEPAL_LENGTH|SEPAL_WIDTH|PETAL_LENGTH|PETAL_WIDTH|VARIETY|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| Setosa|
|         4.9|        3.0|         1.4|        0.2| Setosa|
|         4.7|        3.2|         1.3|        0.2| Setosa|
|         4.6|        3.1|         1.5|        0.2| Setosa|
|         5.0|        3.6|         1.4|        0.2| Setosa|
|         5.4|        3.9|         1.7|        0.4| Setosa|
|         4.6|        3.4|         1.4|        0.3| Setosa|
|         5.0|        3.4|         1.5|        0.2| Setosa|
|         4.4|        2.9|         1.4|        0.2| Setosa|
|         4.9|        3.1|         1.5|        0.1| Setosa|
+------------+-----------+------------+-----------+-------+
only showing top 10 rows



#### Postgres

In [7]:
Postgres_HOST = 'aipost1.cqgrwtaxaib2.us-east-1.rds.amazonaws.com'
Postgres_Port = '5432'
Postgres_DB = 'postgres'
Postgres_TableName = 'IRIS'
Postgres_User_Name = 'B4B4KE4'
Postgres_User_Password = 'AI64732661(('

Postgres_Driver_Name = 'org.postgresql.Driver'
JdbcPostgresUrl = 'jdbc:postgresql://{HOST}:{PORT}/{DB}'.format(HOST=Postgres_HOST,PORT=Postgres_Port,DB=Postgres_DB)

In [8]:
"""TABLEDF = spark.read.format("jdbc").option("url",JdbcPostgresUrl)\
            .option("dbtable",Data_table).option("user",username)\
            .option("password",password)\
            .load()


"""

Postgres_TABLEDF = spark.read.format("jdbc").option("url",JdbcPostgresUrl)\
            .option("dbtable",Postgres_TableName).option("user",Postgres_User_Name)\
            .option("password",Postgres_User_Password)\
            .load()

In [9]:
Postgres_TABLEDF.show()

+------------+-----------+------------+-----------+-------+
|SEPAL_LENGTH|SEPAL_WIDTH|PETAL_LENGTH|PETAL_WIDTH|VARIETY|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| Setosa|
|         4.9|        3.0|         1.4|        0.2| Setosa|
|         4.7|        3.2|         1.3|        0.2| Setosa|
|         4.6|        3.1|         1.5|        0.2| Setosa|
|         5.0|        3.6|         1.4|        0.2| Setosa|
|         5.4|        3.9|         1.7|        0.4| Setosa|
|         4.6|        3.4|         1.4|        0.3| Setosa|
|         5.0|        3.4|         1.5|        0.2| Setosa|
|         4.4|        2.9|         1.4|        0.2| Setosa|
|         4.9|        3.1|         1.5|        0.1| Setosa|
|         5.4|        3.7|         1.5|        0.2| Setosa|
|         4.8|        3.4|         1.6|        0.2| Setosa|
|         4.8|        3.0|         1.4|        0.1| Setosa|
|         4.3|        3.0|         1.1| 

In [11]:
TABLEDF.write.jdbc(
                url=JdbcPostgresUrl, 
                table="test_1", 
                mode="overwrite", 
                properties={
                    "user":Postgres_User_Name, 
                    "password":Postgres_User_Password, 
                    "driver":Postgres_Driver_Name, 
                    "client_encoding":"utf8"
                })

In [12]:
Postgres_TABLEDF = spark.read.format("jdbc").option("url",JdbcPostgresUrl)\
            .option("dbtable","test_1").option("user",Postgres_User_Name)\
            .option("password",Postgres_User_Password)\
            .load().show(55)

+------------+-----------+------------+-----------+----------+
|SEPAL_LENGTH|SEPAL_WIDTH|PETAL_LENGTH|PETAL_WIDTH|   VARIETY|
+------------+-----------+------------+-----------+----------+
|         5.1|        3.5|         1.4|        0.2|    Setosa|
|         4.9|        3.0|         1.4|        0.2|    Setosa|
|         4.7|        3.2|         1.3|        0.2|    Setosa|
|         4.6|        3.1|         1.5|        0.2|    Setosa|
|         5.0|        3.6|         1.4|        0.2|    Setosa|
|         5.4|        3.9|         1.7|        0.4|    Setosa|
|         4.6|        3.4|         1.4|        0.3|    Setosa|
|         5.0|        3.4|         1.5|        0.2|    Setosa|
|         4.4|        2.9|         1.4|        0.2|    Setosa|
|         4.9|        3.1|         1.5|        0.1|    Setosa|
|         5.4|        3.7|         1.5|        0.2|    Setosa|
|         4.8|        3.4|         1.6|        0.2|    Setosa|
|         4.8|        3.0|         1.4|        0.1|    